<a href="https://colab.research.google.com/github/Yadavrahulbabulal/Python_Code/blob/main/Lookalike_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

In [68]:
transactions = transactions.merge(products, on='ProductID', how='left')
data = transactions.merge(customers, on='CustomerID', how='left')

In [69]:
print(transactions.columns)
print(products.columns)
print(data.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'ProductName', 'Category',
       'Price_y'],
      dtype='object')
Index(['ProductID', 'ProductName', 'Category', 'Price'], dtype='object')
Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'ProductName', 'Category',
       'Price_y', 'CustomerName', 'Region', 'SignupDate'],
      dtype='object')


In [70]:
transactions = transactions.merge(products[['ProductID', 'Price']], on='ProductID', how='left')
print(transactions.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x                      ProductName     Category  Price_y  \
0      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
1      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
2      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
3      601.36   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
4      902.04   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   

    Price  
0  300.68  
1  300.68  
2  300.68  
3  300.68  
4  300.68  


In [71]:
transactions['Price'] = transactions['Price'].fillna(products['Price'].median())


In [72]:
if 'Price' in transactions.columns:
    transactions = transactions.drop(columns=['Price'])

transactions = transactions.merge(products[['ProductID', 'Price']], on='ProductID', how='left')

print(transactions.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x                      ProductName     Category  Price_y  \
0      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
1      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
2      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
3      601.36   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
4      902.04   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   

    Price  
0  300.68  
1  300.68  
2  300.68  
3  300.68  
4  300.68  


In [73]:

print("Available columns:", transactions.columns)


if 'Price_x' in transactions.columns and 'Price_y' in transactions.columns and 'Price' in transactions.columns:
    if transactions['Price_x'].equals(transactions['Price_y']) and transactions['Price_x'].equals(transactions['Price']):

        transactions = transactions.drop(columns=['Price_x', 'Price_y'])

        transactions = transactions.rename(columns={'Price': 'ProductPrice'})
    else:
        print("Price columns contain different values. Verify before proceeding!")
else:
    print("Price_x, Price_y, or Price columns do not exist.")


Available columns: Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'ProductName', 'Category',
       'Price_y', 'Price'],
      dtype='object')


In [74]:
print(transactions[['ProductID', 'ProductPrice']].drop_duplicates())


    ProductID  ProductPrice
0        P067        300.68
13       P034        217.05
22       P057        239.70
37       P049        147.95
52       P087        290.17
..        ...           ...
964      P026         64.38
971      P045        481.78
979      P046        366.07
987      P077        265.51
993      P037        459.86

[100 rows x 2 columns]


In [75]:
print(transactions.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue                      ProductName     Category  ProductPrice  
0      300.68  ComfortLiving Bluetooth Speaker  Electronics        300.68  
1      300.68  ComfortLiving Bluetooth Speaker  Electronics        300.68  
2      300.68  ComfortLiving Bluetooth Speaker  Electronics        300.68  
3      601.36  ComfortLiving Bluetooth Speaker  Electronics        300.68  
4      902.04  ComfortLiving Bluetooth Speaker  Electronics        300.68  


In [76]:

revenue_per_product = transactions.groupby('ProductName')['TotalValue'].sum().sort_values(ascending=False)
print(revenue_per_product)


print(revenue_per_product.head(5))


ProductName
ActiveWear Smartwatch          39096.97
SoundWave Headphones           25211.64
SoundWave Novel                24507.90
ActiveWear Jacket              22712.56
ActiveWear Rug                 22314.43
                                 ...   
ComfortLiving Desk Lamp         1694.16
SoundWave Bluetooth Speaker     1223.22
ComfortLiving Rug               1063.81
ActiveWear T-Shirt               795.34
ComfortLiving Laptop             647.76
Name: TotalValue, Length: 66, dtype: float64
ProductName
ActiveWear Smartwatch    39096.97
SoundWave Headphones     25211.64
SoundWave Novel          24507.90
ActiveWear Jacket        22712.56
ActiveWear Rug           22314.43
Name: TotalValue, dtype: float64


In [77]:
customer_features = transactions.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'ProductID': 'nunique'
}).reset_index()

In [78]:
customer_features_normalized = customer_features.drop('CustomerID', axis=1)
customer_features_normalized = (customer_features_normalized - customer_features_normalized.mean()) / customer_features_normalized.std()


In [79]:
similarity_matrix = cosine_similarity(customer_features_normalized)


In [80]:
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])
similarity_df.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.560710,-0.232871,-0.542388,0.533214,-0.362340,0.425749,-0.126259,0.702478,0.343491,...,0.943117,0.708111,0.350032,0.075904,-0.576796,-0.454027,0.407375,0.575206,0.644950,-0.821066
C0002,0.560710,1.000000,0.667108,-0.865484,0.915170,-0.200434,0.839404,-0.663287,0.941406,0.950389,...,0.799450,0.955791,0.863912,-0.632851,-0.813961,-0.221446,0.945113,0.930240,0.987430,-0.925935
C0003,-0.232871,0.667108,1.000000,-0.460825,0.539235,-0.036415,0.527799,-0.568559,0.438427,0.829717,...,0.103343,0.457535,0.631855,-0.727218,-0.361607,0.025884,0.700241,0.525069,0.562845,-0.363571
C0004,-0.542388,-0.865484,-0.460825,1.000000,-0.993363,-0.268157,-0.991074,0.894494,-0.959911,-0.702177,...,-0.702943,-0.944710,-0.973074,0.796384,0.993556,-0.222502,-0.952303,-0.988785,-0.918403,0.777610
C0005,0.533214,0.915170,0.539235,-0.993363,1.000000,0.184247,0.984465,-0.876255,0.972557,0.779409,...,0.721643,0.963393,0.978521,-0.797287,-0.974723,0.145800,0.978551,0.998220,0.951907,-0.816560


In [81]:
lookalike_dict = {}
for customer_id in similarity_df.index:
    similar_customers = similarity_df.loc[customer_id].sort_values(ascending=False)[1:4]
    lookalike_dict[customer_id] = list(zip(similar_customers.index, similar_customers.values))


In [82]:
print(similar_customers)

CustomerID
C0148    0.999536
C0149    0.999492
C0163    0.999043
Name: C0200, dtype: float64


In [83]:
lookalike_dict = {}
for customer_id in similarity_df.index:
    # Sort similarity scores in descending order and select top 3 similar customers
    similar_customers = similarity_df.loc[customer_id].sort_values(ascending=False)[1:4]
    lookalike_dict[customer_id] = list(zip(similar_customers.index, similar_customers.values))

# Print all details of the lookalike_dict
for customer, lookalikes in lookalike_dict.items():
    print(f"CustomerID: {customer}")
    for similar_customer, score in lookalikes:
        print(f"  Similar CustomerID: {similar_customer}, Similarity Score: {score:.4f}")
    print("-" * 50)


CustomerID: C0001
  Similar CustomerID: C0164, Similarity Score: 0.9684
  Similar CustomerID: C0137, Similarity Score: 0.9621
  Similar CustomerID: C0069, Similarity Score: 0.9551
--------------------------------------------------
CustomerID: C0002
  Similar CustomerID: C0029, Similarity Score: 0.9998
  Similar CustomerID: C0031, Similarity Score: 0.9990
  Similar CustomerID: C0094, Similarity Score: 0.9936
--------------------------------------------------
CustomerID: C0003
  Similar CustomerID: C0176, Similarity Score: 0.8906
  Similar CustomerID: C0027, Similarity Score: 0.8636
  Similar CustomerID: C0010, Similarity Score: 0.8297
--------------------------------------------------
CustomerID: C0004
  Similar CustomerID: C0075, Similarity Score: 0.9977
  Similar CustomerID: C0175, Similarity Score: 0.9941
  Similar CustomerID: C0195, Similarity Score: 0.9936
--------------------------------------------------
CustomerID: C0005
  Similar CustomerID: C0058, Similarity Score: 0.9998
  Si

In [84]:
import csv
with open('Lookalike.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['CustomerID', 'Lookalikes'])
    for key, value in lookalike_dict.items():
        writer.writerow([key, value])